# microbit-radio

(We moeten hiervoor nog een andere naam bedenken.)

De onderstaande code is bedoeld voor de microbit die als radio-zender en -ontvanger fungeert van de gateway: de verbinding tussen de gateway en het microbit sensornetwerk.

Deze microbit ontvangt een radio-bericht en stuurt dat hex-gecodeerd door via de seriële verbinding naar de gateway; een hex-gecodeerd bericht ontvangen via de seriële verbinding wordt via de radio verstuurt naar het netwerk.

Deze microbit voert geen controles uit: de berichten worden doorgestuurd zoals ze ontvangen zijn.

In [ ]:
from microbit import *
import radio
import micropython

uart.init(baudrate=9600,rx=pin0,tx=pin1)
micropython.kbd_intr(-1)
in_buf = b''
out_buf = b''
LF = b'\n'

hex = '0123456789ABCDEF'

def nibble(char):
    if char >= ord('0') and char <= ord('9'):
        return char - ord('0')
    elif char >= ord('A') and char <= ord('F'):
        return 10 + char - ord('A')
    elif char >= ord('a') and char <= ord('f'):
        return 10 + char - ord('a')
    else:
        return 0

def hex_to_bytes(data):
    # input bytes or string; only hex chars; even length
    # output bytes
    if type(data) == str:
        data = bytes(data, 'ascii')
    res = []
    assert(len(data) % 2 == 0)
    i = 0
    while i+1 < len(data):
        byte = nibble(data[i]) * 16 + nibble(data[i+1])
        i += 2
        res.append(byte)
    return bytes(res)

def bytes_to_hex(data):
    # input: bytes or int list
    # output: hex bytes
    res = ''
    for i in data:
        res += hex[i // 16] + hex[i % 16]
    return bytes(res, 'ascii')

radio.on()
while True:
    if button_a.was_pressed():
        uart.init(baudrate=115200)  # restore host connection

    if uart.any():
        in_buf = in_buf + uart.read()  # not OK for microbit Python editor?
        
    pos = in_buf.find(LF)
    if pos >= 0 :                                  # complete line found
        radio.send_bytes(hex_to_bytes(in_buf[0:pos]))  # handle first line (excl.LF)
        in_buf = in_buf[pos+1:]                    # and keep the rest (excl. LF)

    if len(out_buf) > 0: # and uart.tx_done():
        w_len = uart.write(out_buf)
        out_buf = out_buf[w_len:]

    rec_bytes = radio.receive_bytes()
    if rec_bytes != None:
        out_buf = out_buf + bytes_to_hex(rec_bytes) + LF    


Commentaar bij bovenstaande:

* `in_buf` is de data die via de uart binnenkomt
* `out_buf` is de data die via de uart verstuurd wordt
* `rec_bytes` is de data van de radio
* de data naar de radio wordt zonder extra buffer verstuurd.
    * er is daarbij geen controle op de lengte van het bericht...
    * de veronderstelling is dat die klein is voor actuator-berichten

Er zijn in de radio-microbit geen print-statements mogelijk, omdat de seriële output gebruikt wordt door de uart! (Eventueel kunnen we later een display gebruiken?)



De regel

```
in_buf = in_buf + uart.read()
```

geeft een foutmelding in de microbit Python editor - die ik niet kan plaatsen. De code werkt wel in de microbit. (Op een of andere manier klopt de type-controle in de editor niet?)


In [4]:
hex_string = '0123456789ABCDEF'
hex_bytes = b'0123456789ABCEDF'

print(hex_string[11]) # returns char (string) 'B'
print(hex_bytes[11])  # returns int 66

B
66
